A classic way to overfit an algorithm is by using lots of features and not a lot of training data. You can find the starter code in feature_selection/find_signature.py. Get a decision tree up and training on the training data, and print out the accuracy.

In [36]:
import pickle
import numpy
numpy.random.seed(42)

The words (features) and authors (labels), already largely processed.
These files should have been created from the previous (Lesson 10)
mini-project.

In [37]:
words_file = "../text_learning/your_word_data.pkl" 
authors_file = "../text_learning/your_email_authors.pkl"
word_data = pickle.load( open(words_file, "r"))
authors = pickle.load( open(authors_file, "r") )

test_size is the percentage of events assigned to the test set (the
remainder go into training)
feature matrices changed to dense representations for compatibility with
classifier functions in versions 0.15.2 and earlier

In [38]:
from sklearn import cross_validation

features_train, features_test, labels_train, labels_test = cross_validation.train_test_split(
    word_data, authors, test_size=0.1, random_state=42)

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                             stop_words='english')
features_train = vectorizer.fit_transform(features_train)
features_test  = vectorizer.transform(features_test).toarray()

classic way to overfit is to use a small number
of data points and a large number of features;
train on only 150 events to put ourselves in this regime

A classic way to overfit an algorithm is by using lots of features and not a lot of training data. You can find the starter code in feature_selection/find_signature.py. Get a decision tree up and training on the training data, and print out the accuracy.
How many training points are there, according to the starter code?

In [39]:
features_train = features_train[:150].toarray()
labels_train   = labels_train[:150]

__What’s the accuracy of the decision tree you just made?__ (Remember, we're setting up our decision tree to overfit -- ideally, we want to see the test accuracy as relatively low.)

In [40]:
from sklearn import tree
from sklearn.metrics import accuracy_score

dt_clf = tree.DecisionTreeClassifier(min_samples_split=40)
dt_clf.fit(features_train, labels_train)

pred = dt_clf.predict(features_test)
accuracy_score(pred, labels_test)

0.81626848691695109

Take your (overfit) decision tree and use the `feature_importances_` attribute to get a list of the relative importance of all the features being used. We suggest iterating through this list (it’s long, since this is text data) and only printing out the feature importance if it’s above some threshold (say, 0.2--remember, if all words were equally important, each one would give an importance of far less than 0.01). __What’s the importance of the most important feature? What is the number of this feature?__

In [41]:
imp = max(dt_clf.feature_importances_)
which_max = dt_clf.feature_importances_.argmax()
#dt_clf.feature_importances_[dt_clf.feature_importances_ == imp]
print(which_max)
print(dt_clf.feature_importances_[which_max])
print(features_train)

21323
0.420772351027
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In order to figure out what words are causing the problem, you need to go back to the TfIdf and use the feature numbers that you obtained in the previous part of the mini-project to get the associated words. You can return a list of all the words in the TfIdf by calling get_feature_names() on it; pull out the word that’s causing most of the discrimination of the decision tree. __What is it? Does it make sense as a word that’s uniquely tied to either Chris Germany or Sara Shackleton, a signature of sorts?__

In [42]:
vectorizer.get_feature_names()[which_max]

u'houectect'

This word seems like an outlier in a certain sense, so let’s remove it and refit. Go back to text_learning/vectorize_text.py, and remove this word from the emails using the same method you used to remove “sara”, “chris”, etc. Rerun vectorize_text.py, and once that finishes, rerun find_signature.py. __Any other outliers pop up? What word is it? Seem like a signature-type word?__ (Define an outlier as a feature with importance >0.2, as before).

In [49]:
g02 = dt_clf.feature_importances_>0.2
print(sum(g02))
print(dt_clf.feature_importances_[g02])
#print(vectorizer.get_feature_names()[dt_clf.feature_importances_[g02]])

2
[ 0.21629799  0.42077235]


Update vectorize_test.py one more time, and rerun. Then run find_signature.py again. Any other important features (importance>0.2) arise? How many? Do any of them look like “signature words”, or are they more “email content” words, that look like they legitimately come from the text of the messages?

What’s the accuracy of the decision tree now? We've removed two "signature words", so it will be more difficult for the algorithm to fit to our limited training set without overfitting. Remember, the whole point was to see if we could get the algorithm to overfit--a sensible result is one where the accuracy isn't that great!